<a href="https://colab.research.google.com/github/Dvidian/Retrieval_Augmented_Generation-RAG-/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/Rag
!ls
%cd Brance
!ls

/content/gdrive/MyDrive/Rag
'AI Applied Researcher Intern Task.pdf'  'Brance task'	  transformers
 Brance					  copyfolder.py
/content/gdrive/MyDrive/Rag/Brance
 Cleanupdocument.txt			     SampleQuestions.xlsx
'KnowledgeDocument(pan_card_services).txt'


In [ ]:
!git clone https://github.com/huggingface/transformers #Run once then comment out
#cd transformers #uncomment for each execution
!pip install transformers Datasets

In [ ]:
#!pip install faiss-gpu
!apt install libomp-dev
!pip install faiss-cpu

#RESTART RUNTIME NOW


In [ ]:
#import transformers
#import datasets
#from datasets import load_dataset


In [ ]:
title = "Pan Card"
article = []
filename = 'Cleanupdocument.txt'
with open(filename,"rb") as f:
  article.append(f.read().decode('utf-8'))
article[0]
#retrievalDocument = load_dataset("text", data_files="Cleanupdocument.txt")
#retrievalDocument['train'][0:50]

"About Pan Card#\n\n What is Pan card?\n\nThe PAN card is a unique ten-digit alphanumeric identification number that is issued by the Income Tax Department of India to track the tax-related transactions of individuals and entities. The PAN card is mandatory for any financial transaction in India, including opening a bank account, buying or selling property, and filing income tax returns.\n\n Who needs a Pan card?\n\nAll individuals/non-individuals (including foreign citizens/entities) earning taxable income in India\xa0must have a PAN card.\n\n Types of PAN cards\n\nIn India, two types of PAN cards are available: e-PAN card and physical PAN card.\n\n1. e-PAN card: An e-PAN card is a digitally-signed PAN card issued in electronic format. It contains the same PAN details as a physical PAN card but is available in a digital format. It can be downloaded online and used as a valid identification document for various purposes. The e-PAN card is usually issued in a PDF format.\n2. Physical PA

# Splitting the text document in 100 word chunk for processing

In [ ]:
passages = []
passage_titles =[]
words = article[0].split()
for i in range(0,len(words),100):
  chunk_words = words[i:i+100]
  chunk = " ".join(chunk_words)
  chunk = chunk.strip()
  if len(chunk) == 0:
    continue
  passage_titles.append(title)
  passages.append(chunk)

chunked_corpus = {'title':passage_titles,'text':passages}
chunked_corpus['text']
print(len(chunked_corpus['text']),len(chunked_corpus['title'][:]))

30 30


In [ ]:

from transformers import DPRContextEncoderTokenizerFast

#Loading the tokenizer
ctx_tokenizer = DPRContextEncoderTokenizerFast.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


In [ ]:
outputs = ctx_tokenizer(
    chunked_corpus["title"],
    chunked_corpus["text"],
    truncation = True,
    padding = "longest",
    return_tensors="pt",
)
input_ids = outputs["input_ids"]
print(input_ids.shape)

torch.Size([30, 205])


In [ ]:
 import torch

 if torch.cuda.is_available():
  device = torch.device("cuda")
  print("GPU is available")
else:
  print("GPU not available")
  device=None

GPU is available


In [ ]:
from transformers import DPRContextEncoder
ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")
ctx_encoder = ctx_encoder.to(device=device)

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-multiset-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.weight', 'ctx_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import math
torch.set_grad_enabled(False)
step = 0
batch_size = 15
num_passages = input_ids.size()[0]
num_batches = math.ceil(num_passages/batch_size)
embeds_batches = []
for i in range(0,num_passages,batch_size):
  #if step%1 == 0 and not step ==0:
    print(step)
    batch_ids = input_ids[i:i+15,:]
    batch_ids = batch_ids.to(device)
    outputs = ctx_encoder(batch_ids,return_dict = True)
    embeddings = outputs['pooler_output']
    embeddings = embeddings.detach().cpu().numpy()
    embeds_batches.append(embeddings)
    step = step + 1

0
1


In [ ]:
import numpy as np

embeddings = np.concatenate(embeds_batches,axis = 0)
print("Size of emebeddings",embeddings.shape)

Size of emebeddings (30, 768)


## FAISS
#Facebook AI Similarity Search

In [ ]:
import faiss

dim = 768

m = 128

index = faiss.IndexHNSWFlat(dim,m,faiss.METRIC_INNER_PRODUCT)

#Building FAISS


In [ ]:

index.train(embeddings)
index.add(embeddings)

In [ ]:
from transformers import DPRQuestionEncoder
q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-multiset-base")
q_encoder = q_encoder.to(device=device)

Some weights of the model checkpoint at facebook/dpr-question_encoder-multiset-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import DPRQuestionEncoderTokenizerFast

q_tokenizer = DPRQuestionEncoderTokenizerFast.from_pretrained("facebook/dpr-question_encoder-multiset-base")

In [ ]:
input_ids = q_tokenizer.encode("Can NRIs get a PAN card?",return_tensors = "pt")
input_ids = input_ids.to(device)
outputs = q_encoder(input_ids)
q_embed = outputs['pooler_output']
q_embed = q_embed.cpu().numpy()
print("query embedding", q_embed.shape)

query embedding (1, 768)


In [ ]:
D,I = index.search(q_embed,k=3)
print("INdices",I,"Inner Product",D)

INdices [[28 16 17]] Inner Product [[83.33659 81.38777 81.36065]]


In [ ]:
 import textwrap
 wrapper = textwrap.TextWrapper(width = 80)

 for i in I[0]:
  print("Index",i)
  title = chunked_corpus['title'][i]
  passage = chunked_corpus['text'][i]
  print("title",title,'\n')
  print("passage:")
  print(wrapper.fill(passage),'\n')

Index 28
title Pan Card 

passage:
NRIs can apply for a PAN card without an Aadhaar Card? They can simply do so by
filling out either of the forms – 49A (for citizens of India) or Form 49AA (for
foreign citizens). Can I apply for a PAN card if I am a non-resident Indian
(NRI)? Yes, as an NRI, you can apply for a PAN card. The process for applying
for a PAN card is the same for both residents and NRIs. However, if you are an
OCI holder or a person of Indian origin who holds foreign citizenship, you will
need to fill Form 49AA 

Index 16
title Pan Card 

passage:
Bank account statement in the country of residence 6. NRE bank account statement
in India 7. Certificate of Residence in India or Residential permit issued by
the State Police Authorities 8. Registration certificate issued by the
Foreigner’s Registration Office showing Indian address 9. Visa granted and Copy
of appointment letter/ contract from Indian Company & Certificate (in original)
of Indian address issued by the employer F

In [ ]:
from datasets import Dataset
import pandas as pd
df = pd.DataFrame(chunked_corpus)
dataset = Dataset.from_pandas(df)
print(dataset)

Dataset({
    features: ['title', 'text'],
    num_rows: 30
})


In [ ]:
embs = []

for i in range(embeddings.shape[0]):
  embs.append(embeddings[i,:])
print(len(embs))

30


In [ ]:
dataset = dataset.add_column("embeddings",embs)
dataset

Dataset({
    features: ['title', 'text', 'embeddings'],
    num_rows: 30
})

In [ ]:
index = faiss.IndexHNSWFlat(dim,m,faiss.METRIC_INNER_PRODUCT)

dataset.add_faiss_index(column = "embeddings",index_name = "embeddings",custom_index = index,faiss_verbose= True)

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['title', 'text', 'embeddings'],
    num_rows: 30
})

In [ ]:
from transformers import RagRetriever

retriever = RagRetriever.from_pretrained(
    "facebook/rag-sequence-nq",
    use_dummy_dataset=False,
    indexed_dataset=dataset,
    index_name="embeddings"
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizerFast'.


In [ ]:
from transformers import RagTokenizer
tokenizer = RagTokenizer.from_pretrained(
    "facebook/rag-sequence-nq"
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

In [ ]:
from transformers import RagSequenceForGeneration
model = RagSequenceForGeneration.from_pretrained(
    "facebook/rag-sequence-nq",
    retriever = retriever
)

Some weights of the model checkpoint at facebook/rag-sequence-nq were not used when initializing RagSequenceForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.bias', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing RagSequenceForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagSequenceForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
question = "Cost"
input_ids = tokenizer.question_encoder(question,return_tensors="pt")["input_ids"]
generated = model.generate(input_ids)
generated_string = tokenizer.batch_decode(generated, skip_special_tokens=True)[0]
print(question,'\n')
print(generated_string)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Cost 

 18 % gst


In [ ]:
def ask_question(question):
  input_ids = tokenizer.question_encoder(question,return_tensors="pt")["input_ids"]
  generated = model.generate(input_ids)
  generated_string = tokenizer.batch_decode(generated, skip_special_tokens=True)[0]
  print(question,'\n')
  print(generated_string)